In [ ]:
%pylab inline --no-import-all

from ctypes import c_float
import sys

# import numpy as np
from astropy.io import fits
from scipy.interpolate import RegularGridInterpolator


from ROOT import gSystem, TFile, TCanvas, TGraphAsymmErrors, TH1D, TH2D, TGraphAsymmErrors, TProfile
import uproot
import matplotlib.pyplot as plt

from root_numpy import hist2array

# from ROOT import VARootIO, VAEffectiveAreaManager, VAEASimpleParameterData
if gSystem.Load("$EVNDISPSYS/lib/libVAnaSum.so"):
    print("Problem loading EventDisplay libraries - please check this before proceeding")
    
# from ROOT import VInstrumentResponseFunctionData

In [ ]:
def remove_duplicities(array, atol):
    i = 0
    while(i < len(array)-1):
        i += 1
        if np.isclose(array[i-1], array[i], atol=atol):
            array = np.delete(array,i-1)
            i -= 1
    return array

def graph_to_array_y(graph):
    y = [g for g in graph.GetY()]
    return y

def graph_to_array_x(graph):
    x = [g for g in graph.GetX()]
    return x

def bin_edges_to_centers(axis):
    # This function assumes bins of equal width
    bin_size = axis[1]-axis[0]
    return np.delete(axis + bin_size/2., len(axis)-1)

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def extract_irf(filename, irf_name, coord_tuple=False, return_irf_axes=False): 
    # Get both the ROOT effective area TTree and the uproot one (much faster)
    effAreaFile = TFile.Open(filename)
    effAreaTree = effAreaFile.Get("fEffArea")
    fast_effArea = uproot.open(filename)['fEffArea']

    # Load parameters from each TTree on arrays with uproot (super fast)
    all_zds = fast_effArea.array('ze')
    all_azs = fast_effArea.array('az')
    all_azMins = fast_effArea.array('azMin')
    all_azMaxs = fast_effArea.array('azMax')
    all_Woffs = fast_effArea.array('Woff')
    all_pedvars = fast_effArea.array('pedvar')
    all_indexs = fast_effArea.array('index')
    all_nbins =  fast_effArea.array('nbins')
    all_rec_nbins =  fast_effArea.array('Rec_nbins')
    
    # If no coord_tuple is provided, extract the IRF over all dimensions
    azs = indexs = pedvars = zds = woffs = []
    if not coord_tuple:
        zds, zes_counts = np.unique(np.round(fast_effArea.array('ze'), decimals=2), return_counts=True)
        azs = np.unique(fast_effArea.array('az'))
        woffs, Woffs_counts = np.unique(np.round(fast_effArea.array('Woff'), decimals=2), return_counts=True)
        pedvars, pedvars_counts = np.unique(np.round(fast_effArea.array('pedvar'), decimals=2), return_counts=True)
        indexs, indexs_counts = np.unique(np.round(fast_effArea.array('index'), decimals=2), return_counts=True)
        # IMPORTANT: Remove duplicities (shouldn't be the case, but the values are not stored properly...)
        # We remove the duplicities from the zenith and pedestal values arrays:
        zds = remove_duplicities(zds, 2.0)
        pedvars = remove_duplicities(pedvars, 0.21)
        if len(all_zds) != len(zds) * len(azs) * len(woffs) * len(pedvars) * len(indexs):
            raise ValueError("Wrong dimensions extracted from IRF cube." +  
                             "Probably due to the rounding applied to the IRF coordinates.")
    # If a specific coord_tuple is provided, only extract the IRFs wihin that range of dimensions
    else:
        if len(coord_tuple) != 5:
            raise ValueError("coord_tuple needs to contain 5 dimensions, in this order: az, index, pedvar, zd and woff")
        else:
            # Get the coordinates to sample:
            azs = coord_tuple[0]
            indexs = coord_tuple[1]
            pedvars = coord_tuple[2]
            zds = coord_tuple[3]
            woffs = coord_tuple[4]
    
    print(azs, indexs, pedvars, zds, woffs)
    
    # For performance, deactivate all branches except the ones needed:
    # Also get the entry with max bins to define the binning in energy
    effAreaTree.SetBranchStatus("*",0) 
    if irf_name == 'eff':
        effAreaTree.SetBranchStatus("e0",1)
        effAreaTree.SetBranchStatus("eff",1)
        entry_with_max_bins = find_nearest(all_nbins, all_nbins.max())
    elif irf_name == 'Rec_eff':
        effAreaTree.SetBranchStatus("Rec_e0",1)
        effAreaTree.SetBranchStatus("Rec_eff",1)
        entry_with_max_bins = find_nearest(all_rec_nbins, all_rec_nbins.max())
    elif irf_name == 'gEffAreaNoTh2MC':
        effAreaTree.SetBranchStatus("gEffAreaNoTh2MC",1)
        entry_with_max_bins = 0
    elif irf_name == 'gEffAreaNoTh2Rec':
        effAreaTree.SetBranchStatus("gEffAreaNoTh2Rec",1)
        entry_with_max_bins = 0
    elif irf_name == 'hEsysMCRelative2D':
        effAreaTree.SetBranchStatus("hEsysMCRelative2D",1)
        entry_with_max_bins = 0
    elif irf_name == 'hAngularDiff_2D':
        effAreaTree.SetBranchStatus("hAngularDiff_2D",1)
        entry_with_max_bins = 0
    else:
        raise Exception("WrongIrfName")
    # Now we know which entry we need to get in order to have a sample IRF
#     sample_irf = sample_energies = []
    for i, entry in enumerate(effAreaTree):
        if i == entry_with_max_bins:
            if irf_name == 'eff':
                sample_irf = [j for j in entry.eff]
                sample_energies = [j for j in entry.e0]
            elif irf_name == 'Rec_eff':
                sample_irf = [j for j in entry.Rec_eff]
                sample_energies = [j for j in entry.Rec_e0]
            elif irf_name == 'gEffAreaNoTh2MC':
                sample_irf = graph_to_array_y(entry.gEffAreaNoTh2MC)
                sample_energies = graph_to_array_x(entry.gEffAreaNoTh2MC)
            elif irf_name == 'gEffAreaNoTh2Rec':
                sample_irf = graph_to_array_y(entry.gEffAreaNoTh2Rec)
                sample_energies = graph_to_array_x(entry.gEffAreaNoTh2Rec)
            elif irf_name == 'hEsysMCRelative2D':
                # Migration vs energy bias and true energy
                sample_irf, axes = hist2array(entry.hEsysMCRelative2D, return_edges=True)
                # Bin edges (one more entry than migra!) for the true energy and 
                # energy bias (Erec/Etrue)
                irf_dimension_1 = bin_edges_to_centers(axes[0])
                irf_dimension_2 = bin_edges_to_centers(axes[1])
            elif irf_name == 'hEsysMCRelative2DNoDirectionCut':
                # Migration vs energy bias and true energy, without direction cut
                sample_irf, axes = hist2array(entry.hEsysMCRelative2DNoDirectionCut, return_edges=True)
                # Bin edges (one more entry than migra!) for the true energy and 
                # energy bias (Erec/Etrue)
                irf_dimension_1 = bin_edges_to_centers(axes[0])
                irf_dimension_2 = bin_edges_to_centers(axes[1])
            elif irf_name == 'hAngularDiff_2D':
                # PSF vs true energy:
                sample_irf, axes = hist2array(entry.hAngularDiff_2D, return_edges=True)
                # Bin edges (one more entry than migra!) for the true energy and 
                # energy bias (Erec/Etrue)
                irf_dimension_1 = bin_edges_to_centers(axes[0])
                irf_dimension_2 = bin_edges_to_centers(axes[1])
                print(irf_dimension_1, irf_dimension_2)
                print("Length: ", len(irf_dimension_1), len(irf_dimension_2))
            else:
                raise Exception("WrongIrfName")
        if i > entry_with_max_bins:
            break
    # Create data container, filled with zeros, containing the required dimensions to store
    # the IRF for a given coord_tuple. Separated between 1 and 2 dimensions:
    if (irf_name == "hEsysMCRelative2D" or irf_name == "hEsysMCRelative2DNoDirectionCut" or 
            irf_name == "hAngularDiff_2D"):
        data = np.zeros((len(irf_dimension_1), len(irf_dimension_2), len(azs), len(indexs), len(pedvars), len(zds), len(woffs)))
    else:
        data = np.zeros((len(sample_irf), len(azs), len(indexs), len(pedvars), len(zds), len(woffs)))
    # Iterate over all IRFs within the file. If the entry i is close to the coordinates within
    # the coord_tuple, then store.
    for i, entry in enumerate(effAreaTree):
        if i%5000 == 0:
            print("{}/{}".format(i, len(all_zds)))
#         if i == 3239:
#             print([j for j in entry.Rec_e0])
        # Parameters within the effective area files show some fluctuation, therefore 
        # we need to use the "isclose".
        if (np.isclose(azs, all_azs[i], atol=0.01).any() and
                np.isclose(indexs, all_indexs[i], atol=0.01).any() and
                np.isclose(pedvars, all_pedvars[i], atol=0.21).any() and
                np.isclose(zds, all_zds[i], atol=2.1).any() and
                np.isclose(woffs, all_Woffs[i], atol=0.05).any()):
            if irf_name == 'eff':
                irf = [j for j in entry.eff]
                energies = [j for j in entry.e0]
            elif irf_name == 'Rec_eff':
                irf = [j for j in entry.Rec_eff]
                energies = [j for j in entry.Rec_e0]
            elif irf_name == 'gEffAreaNoTh2MC':
                irf = graph_to_array_y(entry.gEffAreaNoTh2MC)
                energies = graph_to_array_x(entry.gEffAreaNoTh2MC)
            elif irf_name == 'gEffAreaNoTh2Rec':
                irf = graph_to_array_y(entry.gEffAreaNoTh2Rec)
                energies = graph_to_array_x(entry.gEffAreaNoTh2Rec)
            elif irf_name == 'hEsysMCRelative2D':
                irf = hist2array(entry.hEsysMCRelative2D)
            elif irf_name == 'hEsysMCRelative2DNoDirectionCut':
                irf = hist2array(entry.hEsysMCRelative2DNoDirectionCut)
            elif irf_name == 'hAngularDiff_2D':
                irf = hist2array(entry.hAngularDiff_2D)
            else:
                raise Exception("WrongIrfName")
            # We separate now 1D and 2D irfs:
            # 2D IRFs:
            if (irf_name == "hEsysMCRelative2D" or irf_name == "hEsysMCRelative2DNoDirectionCut" or 
                    irf_name == "hAngularDiff_2D"):
                if np.shape(irf) == np.shape(sample_irf):
                    try:
                        data[:,:, find_nearest(azs, all_azs[i]), find_nearest(indexs, all_indexs[i]), find_nearest(pedvars, all_pedvars[i]), 
                             find_nearest(zds, all_zds[i]), find_nearest(woffs, all_Woffs[i])] = irf
                    except:
                        print("Unexpected error:", sys.exc_info()[0])
                        print("Entry number ", i)
                        raise
                else:
                    print("2D IRFs of variable size...")
                    raise
            else:
                # 1D IRFs:
                # In case the extracted IRF has less bins than the "sample" one, pad it with zeros:
                if len(irf) < len(sample_irf):
                    new_irf = np.zeros(len(sample_energies))
                    for j, ener in enumerate(energies):
                        new_irf[find_nearest(sample_energies, ener)] = irf[j]
                    irf = new_irf
                try:
                    data[:, find_nearest(azs, all_azs[i]), find_nearest(indexs, all_indexs[i]), find_nearest(pedvars, all_pedvars[i]), 
                         find_nearest(zds, all_zds[i]), find_nearest(woffs, all_Woffs[i])] = irf
                except:
                    print("Unexpected error:", sys.exc_info()[0])
                    print("Entry number ", i)
                    print(energies, irf)
                    raise 
    if return_irf_axes:
        if (irf_name == "hEsysMCRelative2D" or irf_name == "hEsysMCRelative2DNoDirectionCut" or 
                irf_name == "hAngularDiff_2D"):
            return data, [np.array(irf_dimension_1), np.array(irf_dimension_2), azs, indexs, pedvars, zds, woffs]
        else:
            return data, [np.array(sample_energies), azs, indexs, pedvars, zds, woffs]
    else:
        return data


def duplicate_dimension(data, axis):
    # This function duplicates a single axis, assuming it's length is 1.
    current_shape = np.shape(data)
    corrected_shape = [2 if i == axis else k for i, k in enumerate(current_shape)]
    print(current_shape, corrected_shape)
    new_data = np.zeros(corrected_shape)
    sl = slice(None, None, None)
    new_data[[0 if i == axis else sl for i, k in enumerate(current_shape)]] = \
        data[[0 if i == axis else sl for i, k in enumerate(current_shape)]]
    new_data[[1 if i == axis else sl for i, k in enumerate(current_shape)]] = \
        data[[0 if i == axis else sl for i, k in enumerate(current_shape)]]
    return new_data


def duplicate_dimensions(data):
    # This function duplicates all axes, one by one, when their size is 1.
    new_data = data
    current_shape = np.shape(new_data)
    for i, dim in enumerate(current_shape):
        if (dim == 1):
            new_data = duplicate_dimension(new_data, i)
    return new_data

In [ ]:
# Let's play around a bit with v501_rd effective area files (after merging, with surviving histograms)
# effAreaFileName = '/home/thassan/VERITAS/DL3/DL3_for_eventDisplay/EffectiveAreas_v501_rd/effArea-v501-TL5035MA20-auxv01-CARE_June1702-Cut-NTel4-PointSource-TMVA-BDT-Preselection-GEOID3-V6-ATM61-T1234.root'
effAreaFileName = '/home/thassan/VERITAS/DL3/DL3_for_eventDisplay/EffectiveAreas_v501_rd/effArea-v501-TL5035MA20-auxv01-CARE_June1702-Cut-NTel2-PointSource-TMVA-BDT-Preselection-GEOID0-V6-ATM61-T1234.root'

effAreaFile = TFile.Open(effAreaFileName)
effAreaTree = effAreaFile.Get("fEffArea")
fast_effArea = uproot.open(effAreaFileName)['fEffArea']

In [ ]:
# Grab some values from the first element of the file:
for i, entry in enumerate(effAreaTree):
    if i == 0:
        e0 = [i for i in entry.e0]
    else:
        break
print(e0)

In [ ]:
zes, zes_counts = np.unique(np.round(fast_effArea.array('ze'), decimals=2), return_counts=True)
azs = np.unique(fast_effArea.array('az'))
azMins = np.unique(fast_effArea.array('azMin'))
azMaxs = np.unique(fast_effArea.array('azMax'))
Woffs, Woffs_counts = np.unique(np.round(fast_effArea.array('Woff'), decimals=2), return_counts=True)
noises, noises_counts = np.unique(np.round(fast_effArea.array('noise'), decimals=2), return_counts=True)
pedvars, pedvars_counts = np.unique(np.round(fast_effArea.array('pedvar'), decimals=2), return_counts=True)
indexs, indexs_counts = np.unique(np.round(fast_effArea.array('index'), decimals=2), return_counts=True)
all_nbins =  fast_effArea.array('nbins')
all_rec_nbins =  fast_effArea.array('Rec_nbins')

# IMPORTANT: Remove duplicities (shouldn't be the case, but the values are not stored properly...)
# We remove the duplicities from the zenith and pedestal values arrays:
zes = remove_duplicities(zes, 2.0)
pedvars = remove_duplicities(pedvars, 0.21)

In [ ]:
# Load parameters from each TTree on arrays with uproot (super fast)
all_zds = fast_effArea.array('ze')
all_azs = fast_effArea.array('az')
all_azMins = fast_effArea.array('azMin')
all_azMaxs = fast_effArea.array('azMax')
all_Woffs = fast_effArea.array('Woff')
all_pedvars = fast_effArea.array('pedvar')
all_indexs = fast_effArea.array('index')

In [ ]:
print(len(all_zds), len(zes) * len(azs) * len(Woffs) * len(pedvars) * len(indexs))

In [ ]:
print(zes, azs, Woffs, pedvars, indexs)

In [ ]:
pedvars

In [ ]:
# We remove the duplicities from the zenith and pedestal values arrays:
zes = remove_duplicities(zes, 2.0)
pedvars = remove_duplicities(pedvars, 0.21)

The order we will use is:
* Azimuth
* Index
* Pedestal variance
* Zenith
* Camera offset (Woff)

In [ ]:
# Just grab some effective areas as a function of the zenith:
#   - First bin in azimuth, index of 1.5, pedvar of 4.4 and standard wobble distance
dimensions_to_sample = [[0], indexs, pedvars, zes, [0.5]]

In [ ]:
irf_data, irf_axes = extract_irf('/home/thassan/VERITAS/DL3/DL3_for_eventDisplay/EffectiveAreas_v501_rd/effArea-v501-TL5035MA20-auxv01-CARE_June1702-Cut-NTel2-PointSource-TMVA-BDT-Preselection-GEOID0-V6-ATM61-T1234.root',
                       'eff', return_irf_axes = True)
# irf_data = extract_irf('/home/thassan/VERITAS/DL3/DL3_for_eventDisplay/EffectiveAreas_v501_rd/effArea-v501-TL5035MA20-auxv01-CARE_June1702-Cut-NTel2-PointSource-TMVA-BDT-Preselection-GEOID0-V6-ATM61-T1234.root',
#                        'Rec_eff', dimensions_to_sample )
# irf_data = extract_irf('/home/thassan/VERITAS/DL3/DL3_for_eventDisplay/EffectiveAreas_v501_rd/effArea-v501-TL5035MA20-auxv01-CARE_June1702-Cut-NTel2-PointSource-TMVA-BDT-Preselection-GEOID0-V6-ATM61-T1234.root',
#                        'gEffAreaNoTh2MC', dimensions_to_sample )
# irf_data, dims = extract_irf('/home/thassan/VERITAS/DL3/DL3_for_eventDisplay/EffectiveAreas_v501_rd/effArea-v501-TL5035MA20-auxv01-CARE_June1702-Cut-NTel2-PointSource-TMVA-BDT-Preselection-GEOID0-V6-ATM61-T1234.root',
#                        'hEsysMCRelative2D', dimensions_to_sample, return_irf_axes=True)
print(np.shape(irf_data))
print(irf_axes)

In [ ]:
irf_data, dims = extract_irf('/home/thassan/VERITAS/DL3/DL3_for_eventDisplay/EffectiveAreas_v501_rd/effArea-v501-TL5035MA20-auxv01-CARE_June1702-Cut-NTel2-PointSource-TMVA-BDT-Preselection-GEOID0-V6-ATM61-T1234.root',
                             'hEsysMCRelative2D', dimensions_to_sample, return_irf_axes=True)

# For 2D IRFs:
fig = plt.figure(figsize=(10,15))
ax = []
# Loop over the zenith dimension:
nplot = 420
for i in np.arange(np.shape(irf_data)[5]):
    nplot += 1
    ax.append(fig.add_subplot(nplot))    # The big subplot
    ax[i].imshow(np.flip(irf_data[:,:,0,0,0,i,0].transpose(), axis=0), vmin=irf_data[:,:,0,0,0,i,0].min(), vmax=irf_data[:,:,0,0,0,i,0].max(),
                extent=[dims[0].min(), dims[0].max(), dims[1].min(), dims[1].max()])
    ax[i].set_title('Zd = {} deg'.format(zes[i]))
# ax.grid(color='gray', linestyle='-', linewidth=2)
# ax.set_xlabel('# bin')
# ax.set_yscale("log", nonposy='clip')
# ax.set_ylabel('Effective area')
fig.show()

In [ ]:
print(dims)

Effective area vs zenith:
-----------------------------

In [ ]:
irf_data, axes = extract_irf('/home/thassan/VERITAS/DL3/DL3_for_eventDisplay/EffectiveAreas_v501_rd/effArea-v501-TL5035MA20-auxv01-CARE_June1702-Cut-NTel2-PointSource-TMVA-BDT-Preselection-GEOID0-V6-ATM61-T1234.root',
                       'eff', dimensions_to_sample, return_irf_axes = True )

fig = plt.figure(figsize=(14,10))
ax = fig.add_subplot(111)
for i in np.arange(np.shape(irf_data)[4]):
    ax.plot(axes[0], irf_data[:,0,0,0,i,0], marker='o', label='Zd = {} deg'.format(zes[i]))
ax.grid(color='gray', linestyle='-', linewidth=2)
ax.set_xlabel('log$_{10}$( Energy) TeV')
ax.set_yscale("log", nonposy='clip')
ax.set_ylabel('Effective area')
ax.legend()
fig.show()

Effective area vs index:
---------------------------
(Should not vary, because the effective area is vs true energy)

In [ ]:
fig = plt.figure(figsize=(14,10))
ax = fig.add_subplot(111)
for i in np.arange(np.shape(irf_data)[2]):
    ax.plot(axes[0], irf_data[:,0,i,0,0,0]/irf_data[:,0,0,0,0,0], marker='o', label='Index = {}'.format(indexs[i]))
ax.grid(color='gray', linestyle='-', linewidth=2)
ax.set_xlabel('log$_{10}$( Energy) TeV')
# ax.set_yscale("log", nonposy='clip')
ax.set_ylabel('Effective area ratio')
ax.legend()
fig.show()

Effective area vs index (vs E$_{est}$):
---------------------------
(Should vary, because the effective area is vs estimated energy)

In [ ]:
irf_data, axes = extract_irf('/home/thassan/VERITAS/DL3/DL3_for_eventDisplay/EffectiveAreas_v501_rd/effArea-v501-TL5035MA20-auxv01-CARE_June1702-Cut-NTel2-PointSource-TMVA-BDT-Preselection-GEOID0-V6-ATM61-T1234.root',
                       'Rec_eff', dimensions_to_sample, return_irf_axes = True )
fig = plt.figure(figsize=(14,10))
ax = fig.add_subplot(111)
for i in np.arange(np.shape(irf_data)[2]):
    ax.plot(axes[0], irf_data[:,0,i,0,0,0]/irf_data[:,0,0,0,0,0], marker='o', label='Index = {}'.format(indexs[i]))
ax.grid(color='gray', linestyle='-', linewidth=2)
ax.set_xlabel('log$_{10}$( Energy) TeV')
# ax.set_yscale("log", nonposy='clip')
ax.set_ylabel('Effective area')
ax.legend()
fig.show()

Some tests I was running, playing around with the interpolator.

In [ ]:
# Evaluate the interpolating function at zenith = 10 deg
# interpolated_eff_area = np.zeros([len(dims[0]), len(dims[1])])
# for i, x in enumerate(dims[0]):
#     for j, y in enumerate(dims[1]):
#         interpolated_eff_area[i,j] = my_interpolating_function([x,y, 1.872, 5.7, 35.])

In [ ]:
# shap = np.shape(interpolated_eff_area)

In [ ]:
# XX,YY=np.meshgrid(dims[0],dims[1])
# fast_interpolated_eff_area = my_interpolating_function((XX,YY,1.872, 5.7, 35.))

In [ ]:
# fig = plt.figure(figsize=(10,15))
# # Loop over the zenith dimension:
# nplot = 420
# ax1 = fig.add_subplot(121)    # The big subplot
# ax1.imshow(np.flip(interpolated_eff_area.transpose(), axis=0), vmin=interpolated_eff_area.min(), vmax=interpolated_eff_area.max(),
#             extent=[dims[0].min(), dims[0].max(), dims[1].min(), dims[1].max()])
# ax1.set_title('Slow interpolation')

# ax2 = fig.add_subplot(122)    # The big subplot
# ax2.imshow(np.flip(fast_interpolated_eff_area, axis=0), vmin=fast_interpolated_eff_area.min(), vmax=fast_interpolated_eff_area.max(),
#             extent=[dims[0].min(), dims[0].max(), dims[1].min(), dims[1].max()])
# ax2.set_title('Fast interpolation')

In [ ]:
# np.arange(shap[0])

In [ ]:
# for i in np.arange(shap[0]):
#     for j in np.arange(shap[1]):
#         if interpolated_eff_area[i,j] != fast_interpolated_eff_area[i,j]:
#             print("They are different!!")

In [ ]:
# if interpolated_eff_area == my_interpolating_function((XX,YY,1.872, 5.7, 35.)):
#     print("Identical result 1000 times faster!")

In [ ]:
# XX,YY=np.meshgrid(dims[0],dims[1])
# np.shape(my_interpolating_function((XX,YY,1.872, 5.7, 35.)))

In [ ]:
# fig = plt.figure(figsize=(14,10))
# ax = fig.add_subplot(111)
# for i in np.arange(2):
#     ax.plot(np.arange(len(irf_data[:,0,0,0,i,0])), irf_data[:,0,0,0,i,0], marker='o', label='Zd = {} deg'.format(zes[i]))

# ax.plot(np.arange(len(irf_data[:,0,0,0,i,0])), interpolated_eff_area, marker='o', label='Interp Zd = {} deg'.format(10.))
# ax.grid(color='gray', linestyle='-', linewidth=2)
# ax.set_xlabel('# bin')
# ax.set_yscale("log", nonposy='clip')
# ax.set_ylabel('Effective area')
# ax.legend()
# fig.show()

The final IRF interpolator will have this shape:
* Inputs:
    * Effective area file, containing all IRFs in the form $A_{i,j,k,l,m}$ where A can be any IRF and $i..m$ the dimensions within the file.
    * The value $X = [x_i, x_j, x_k, x_l, x_m]$, a point in the 5D "hyper cube" of IRFs, to be interpolated.
    * The dimensions will be:
        * Azimuth
        * Power-law index
        * Pedestal variance
        * Zenith
        * Camera offset (Woff)
* The steps that need to be taken are:
    * First read the IRF file: 
        * Load range of all available dimensions $[D_{min}, ..., D_{max}]_{i,j,k,l,m}$ within the file (points corresponding to each IRF)
        * Load a 5D cube of a given IRF, as a function of all dimensions 
    * Check if $X \in D$. If yes, continue. If not... New IRFs need to be computed.
    * Interpolate the IRF:
        * Make sure that there is no "1D" -> Needs to be at least 2 to use the regular grid interpolator

In [ ]:
loaded_irf_name = False
loaded_irf_filename = False
loaded_interpolator = False
loaded_irf_data = False
loaded_irf_axes = False

def interpolate_irf(filename, irf_name, coordinate):
    compute = True
    # If the table for the given file and IRF has been already loaded, do not load again:
    if filename == loaded_irf_filename and irf_name == loaded_irf_name:
        compute = False
        
    # List of IRFs that are currently implemented. 
    implemented_irf_names = ['eff', 'Rec_eff', 'gEffAreaNoTh2MC', 'gEffAreaNoTh2Rec', 
                             'hEsysMCRelative2D', 'hEsysMCRelative2DNoDirectionCut', 'hAngularDiff_2D']
    if irf_name not in implemented_irf_names:
        print("The irf you entered: {}  is either wrong or not implemented.".format(irf_name))
        raise Exception("WrongIrfName")
    # Extract all IRFs within "filename" of type "irf_name":
    irf_data, irf_axes = extract_irf(filename, irf_name, return_irf_axes = True)
    
    # This is an important technical step: the regular grid interpolator does not accept interpolating on
    # a dimension with size = 1.
    # Make sure that there are no size 1 dimensions. Do the same with the axes:
    irf_data = duplicate_dimensions(irf_data)
    for i, axis in enumerate(irf_axes):
        if len(axis) == 1:
            irf_axes[i] = np.concatenate((axis.flatten(), axis.flatten()+0.01), axis = None)
    
    # Create the interpolator
    loaded_interpolator = RegularGridInterpolator(irf_axes, irf_data)
    # The interpolation is slightly different for 1D or 2D IRFs. We do both cases separated:
    if (irf_name == "hEsysMCRelative2D" or irf_name == "hEsysMCRelative2DNoDirectionCut" or 
        irf_name == "hAngularDiff_2D"):
        # In this case, the interpolator needs to interpolate over 2 dimensions:
        XX,YY=np.meshgrid(irf_axes[0],irf_axes[1])
        loaded_interpolator = interpolator((XX,YY,coordinate[0], coordinate[1], coordinate[2], 
                                                      coordinate[3], coordinate[4]))
        return interpolated_irf, [irf_axes[0],irf_axes[1]]
    else:
        # In this case, the interpolator needs to interpolate only over 1 dimension:
        XX,YY=np.meshgrid(irf_axes[0],irf_axes[1])
        loaded_interpolator = interpolator((irf_axes[0],coordinate[0], coordinate[1], coordinate[2], 
                                                      coordinate[3], coordinate[4]))
        return interpolated_irf, [irf_axes[0]]

In [ ]:
irf_data, dims = interpolate_irf('/home/thassan/VERITAS/DL3/DL3_for_eventDisplay/EffectiveAreas_v501_rd/effArea-v501-TL5035MA20-auxv01-CARE_June1702-Cut-NTel2-PointSource-TMVA-BDT-Preselection-GEOID0-V6-ATM61-T1234.root',
                                 'gEffAreaNoTh2MC', [0, 2.5999999, 3.96000004, 25., 0.5])

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
ax.plot(dims[0], irf_data, marker='o')
ax.grid(color='gray', linestyle='-', linewidth=2)
ax.set_xlabel('log$_{10}$( Energy) TeV')
ax.set_yscale("log", nonposy='clip')
ax.set_ylabel('Effective area')
ax.legend()
fig.show()

In [ ]:
dims[0]

In [ ]:
irf_data, dims = interpolate_irf('/home/thassan/VERITAS/DL3/DL3_for_eventDisplay/EffectiveAreas_v501_rd/effArea-v501-TL5035MA20-auxv01-CARE_June1702-Cut-NTel2-PointSource-TMVA-BDT-Preselection-GEOID0-V6-ATM61-T1234.root',
                                 'hEsysMCRelative2D', [0, 2.5999999, 3.96000004, 25., 0.5])
fig = plt.figure(figsize=(10,15))
plt.imshow(np.flip(irf_data, axis=0), vmin=irf_data.min(), vmax=irf_data.max(),
           extent=[dims[0].min(), dims[0].max(), dims[1].min(), dims[1].max()])

In [ ]:
# Bin size in energy
migra_bin_size = dims[1][1]-dims[1][0]
norm_irf_data = np.array(irf_data)
# Renormalize energy bias plot. For each slice in energy (dim[0]) normalize the curve to have integral 1.
for i in np.arange(len(dims[0])):
    if np.sum(norm_irf_data[:,i]) > 0.:
        norm_irf_data[:,i] = norm_irf_data[:,i]/np.sum(norm_irf_data[:,i]*migra_bin_size)

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
for i in np.arange(len(dims[0])):
    if i%4 == 0:
        ax.plot(dims[1], irf_data[:,i], marker='o', label='Energy = {:.2f} TeV'.format(10**dims[0][i]))
ax.grid(color='gray', linestyle='-', linewidth=2)
ax.set_xlabel('$E_{Reco} / E_{True}$')
# ax.set_xscale("log", nonposx='clip')
# ax.set_yscale("log", nonposy='clip')
ax.set_ylabel('Migration')
ax.legend()
fig.show()

In [ ]:
def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
for i in np.arange(len(dims[0])):
    if i%4 == 0 and i > 8:
#     if 10**dims[0][i] > 50.:
        ax.plot(dims[1], smooth(norm_irf_data[:,i], 4), marker='o', label='Energy = {:.2f} TeV'.format(10**dims[0][i]))
ax.grid(color='gray', linestyle='-', linewidth=2)
ax.set_xlabel('$E_{Reco} / E_{True}$')
# ax.set_xscale("log", nonposx='clip')
# ax.set_yscale("log", nonposy='clip')
ax.set_ylabel('Migration')
ax.legend()
fig.show()

In [ ]:
# Testing different energy migration smoothings:
fig = plt.figure(figsize=(15,15))
ax = []
smooths = [3, 5, 7, 10]
nplot = 420
for ip, ismooth in enumerate(smooths):
    nplot += 1
    ax.append(fig.add_subplot(nplot))
    for i in np.arange(len(dims[0])):
        if i%4 == 0 and i > 8 and np.sum(norm_irf_data[:,i]) > 0.:
    #     if 10**dims[0][i] > 50.:
            ax[ip].plot(dims[1], smooth(norm_irf_data[:,i], ismooth), marker='.', label='Energy = {:.2f} TeV'.format(10**dims[0][i]))
            ax[ip].set_title('Smoothing {}'.format(ismooth))
            ax[ip].legend()

In [ ]:
print(dims[0], [np.sum(irf_data[:,i]) for i in np.arange(len(dims[0]))])

In [ ]:
print(dims[1])

In [ ]:
irf_data, dims = interpolate_irf('/home/thassan/VERITAS/DL3/DL3_for_eventDisplay/EffectiveAreas_v501_rd/effArea-v501-TL5035MA20-auxv01-CARE_June1702-Cut-NTel2-PointSource-TMVA-BDT-Preselection-GEOID0-V6-ATM61-T1234.root',
                                 'hAngularDiff_2D', [0, 2.5999999, 3.96000004, 25., 0.5])
fig = plt.figure(figsize=(10,15))
plt.imshow(np.flip(irf_data, axis=0), vmin=irf_data.min(), vmax=irf_data.max(),
           extent=[dims[0].min(), dims[0].max(), dims[1].min(), dims[1].max()])

In [ ]:
class Irf_interpolator:
    def __init__(self, filename):
        import os.path
        if os.path.isfile(filename):
            self.filename = filename
        else:
            raise Exception("Wrong Effective area file")

    def set_irf(self, irf_name):
        implemented_irf_names = ['eff', 'Rec_eff', 'gEffAreaNoTh2MC', 'gEffAreaNoTh2Rec', 
                                  'hEsysMCRelative2D', 'hEsysMCRelative2DNoDirectionCut', 'hAngularDiff_2D']
        if irf_name in implemented_irf_names:
            self.irf_name = irf_name
            self.__load_irf(irf_name)
        else:
            print("The irf you entered: {}  is either wrong or not implemented.".format(irf_name))
            raise Exception("WrongIrfName")
            
    def __load_irf(self, irf_name):
        irf_data, irf_axes = extract_irf(self.filename, self.irf_name, return_irf_axes = True)
        # This is an important technical step: the regular grid interpolator does not accept 
        # interpolating on a dimension with size = 1.
        # Make sure that there are no size 1 dimensions. Do the same with the axes:
        irf_data = duplicate_dimensions(irf_data)
        # Also the coordinates of the axes need to be in increasing order.
        for i, axis in enumerate(irf_axes):
            if len(axis) == 1:
                irf_axes[i] = np.concatenate((axis.flatten(), axis.flatten()+0.01), axis = None)
        self.irf_data = irf_data
        self.irf_axes = irf_axes
        self.interpolator = RegularGridInterpolator(self.irf_axes, self.irf_data)
        
    def interpolate(self, coordinate):
        # The interpolation is slightly different for 1D or 2D IRFs. We do both cases separated:
        if (self.irf_name == "hEsysMCRelative2D" or 
#             self.irf_name == "hAngularDiff_2D" or    # Angular Resolution needs to be generated and validated!!
            self.irf_name == "hEsysMCRelative2DNoDirectionCut"
           ):
            # In this case, the interpolator needs to interpolate over 2 dimensions:
            XX,YY=np.meshgrid(self.irf_axes[0],self.irf_axes[1])
            interpolated_irf = self.interpolator((XX,YY,coordinate[0], coordinate[1], coordinate[2], 
                                             coordinate[3], coordinate[4]))
            return interpolated_irf, [self.irf_axes[0],self.irf_axes[1]]
        else:
            # In this case, the interpolator needs to interpolate only over 1 dimension (true energy):
            interpolated_irf = self.interpolator((self.irf_axes[0],coordinate[0], coordinate[1], 
                                             coordinate[2], coordinate[3], coordinate[4]))
            return interpolated_irf, [self.irf_axes[0]]

In [ ]:
irf_generator = Irf_interpolator('/home/thassan/VERITAS/DL3/DL3_for_eventDisplay/EffectiveAreas_v501_rd/effArea-v501-TL5035MA20-auxv01-CARE_June1702-Cut-NTel2-PointSource-TMVA-BDT-Preselection-GEOID0-V6-ATM61-T1234.root')
irf_generator.set_irf('eff')

In [ ]:
irf, axis = irf_generator.interpolate([0, 2.5999999, 3.96000004, 10, 0.5])

In [ ]:
axis[0]

In [ ]:
fig = plt.figure(figsize=(14,10))
ax = fig.add_subplot(111)
for zenith in np.arange(0, 60, 10):
    irf, axis = irf_generator.interpolate([0, 2.5999999, 3.96000004, zenith, 0.5])
    ax.plot(axis[0], irf, marker='o', label='Zd = {} deg'.format(zenith))
ax.grid(color='gray', linestyle='-', linewidth=2)
ax.set_xlabel('log$_{10}$( Energy) TeV')
ax.set_yscale("log", nonposy='clip')
ax.set_ylabel('Effective area')
ax.legend()
fig.show()